<a href="https://colab.research.google.com/github/jugernaut/MACTI-analisisnumerico/blob/josue/09_ProgramacionParalelo/CPU/02_CudaNumbaParalelo_SCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Programación en Paralelo</i></h1>
  </font>
  <font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>Profesor: M. en C. Miguel Angel Pérez León</i></h5>
    <h5 align="center"><i>Ayudante: Jesús Iván Coss Calderón</i></h5>
    <h5 align="center"><i>Ayudante: Mario Arturo Nieto Butron</i></h5>
  <h5 align="center"><i>Materia: Seminario de programación en paralelo</i></h5>
  </font>

# Introducción

Todos los algoritmos vistos en el curso se han descrito de manera [secuencial](https://desarrolloweb.com/articulos/2199.php); sin embargo,muchos de ellos pueden ser redefinidos en su forma en **paralelo**.

¿Por qué nos debería interesar programar en paralelo?, bueno la respuesta es muy sencilla, la gran mayoría (si no es que todas) de las aplicaciones que usamos actualmente son desarrolladas usando alguna técnica de programación en paralelo.

¿Cuál es la principal ventaja de la programación en paralelo?, **la velocidad con la que este se ejecuta**, es decir el tiempo de ejecución. Un algoritmo en paralelo, debido a que distribuye los cálculos (*FLOP's*) entre los distintos dispositivos de cómputo tiende a reducir el tiempo de ejecución.

*CUDA* (estándar de programación en paralelo basado en el lenguaje *C/C++*) tiene un modelo de ejecución diferente al modelo secuencial tradicional utilizado para programar usando los *CPU's* disponibles. En *CUDA*, el código que escribes será ejecutado por varios subprocesos a la vez (a menudo cientos o miles dentro de la *GPU*), a diferencia de la programación secuencial en la cual cada [proceso](http://tecnologiasifa4b.blogspot.com/2015/08/proceso-computacional.html#:~:text=Un%20proceso%20computacional%20es%20u,de%20recursos%20del%20sistema%20asociados.&text=La%20ejecuci%C3%B3n%2C%20desde%20un%20proceso,la%20creaci%C3%B3n%20de%20otro%20proceso.) es ejecutado uno a la vez por cada *CPU*. Su solución se modelará definiendo una jerarquía de subprocesos de cuadrícula, bloques y subprocesos.

Una de las bibliotecas que da soporte a *CUDA* para *Python* recibe el nombre de *Numba* ([envoltorio](https://es.wikipedia.org/wiki/Wrapper) de *CUDA* para python). Mediante Numba podemos acceder a las instalaciones para declarar y administrar las características que ofrece *CUDA*. Las instalaciones y prestaciones son muy similares a las expuestas por el lenguaje *C/C++* para *CUDA* de *Nvidia*.

## *CPU* v.s. *GPU*

Actualmente los dispositivos de cómputo contienen al menos un *CPU* y dentro de este *CPU* pueden estar contenidos varios núcleos, lo que permite el desarrollo de algoritmos en paralelo.

De igual manera, la mayoría de los dispositivos de cómputo contienen al menos un *GPU* y dentro de este *GPU* pueden existir varios núcleos, la principal diferencia entre ambos (*CPU* y *GPU*) es el propósito para el cuál fueron diseñados

Para fines prácticos (y del curso) podemos pensar que la diferencia principal entre una *CPU* (unidad de procesamiento central) y una *GPU* (**unidad de procesamiento gráfico**) radica en que un *CPU* es un dispositivo de cómputo de propósito general, puede realizar cualquier tipo de cómputo que se le asigne.

Por otro lado un *GPU* esta diseñada para el procesamiento gráfico, lo que significa que la forma en la que procesa información **esta optimizada para trabajar con matrices y vectores**.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/desarrollo/Figuras/MonteCarlo/cpu-vs-gpu.jpg?raw=1" width="600">
</center>

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/desarrollo/Figuras/MonteCarlo/mejor.png?raw=1" width="600">
</center>

## Ley de Moore

El número de transistores por unidad de superficie en circuitos integrados, se duplicara cada año. En consecuencia, la velocidad de cómputo relacionada directamente al hardware se vera duplicada cada año.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/desarrollo/Figuras/MonteCarlo/more2.jpg?raw=1" width="800">
</center>

## ¿Qué es *Numba*?

Según la documentación de *Numba*, "*Numba* es un compilador justo a tiempo para *Python* que funciona mejor en código que usa matrices, funciones de *Numpy* y ciclos.

En otras palabras, *Numba* es una paquetería (igual que *Numpy* o *Matplotlib*) que nos ayuda a que nuestros algoritmos se ejecuten de forma optimizada y en particular nos permite tener acceso a los *GPU's* disponibles en el hardware que estemos ejecutando nuestros algoritmos.

La forma más común de usar *Numba* es a través de su colección de *decoradores* que se pueden aplicar a sus funciones para indicarle a *Numba* que las compile. Cuando se realiza una llamada a una función decorada de *Numba*, **se compila en código máquina** "justo a tiempo" para su ejecución y todo o parte de su código puede ejecutarse posteriormente a la ¡velocidad del código máquina nativo!.

Para nuestros propósitos de hoy, *Numba* es un paquete de *Python* que le permite escribir código *Python* para *GPU*, y también puede usarse para programación en paralelo mediante *CPU's*.

## Dependencia Funcional

Existen muchas formas para poder identificar si existe la posibilidad de emplear alguna técnica de paralelización algun algoritmo.

Sin embargo la forma más sencilla de saber cuando NO ES POSIBLE programar algún algoritmo en su equivalente en paralelo, es mediante las dependencias funcionales.

La idea de **dependencia funcional** es la generalización del concepto de **dependencia lineal**. Se dice que un conjunto de funciones es **funcionalmente dependiente** si existe una relación funcional entre ellas, en otras palabras cuando alguna de las funciones del conjunto es expresable como función de las otras funciones definidas previamente dentro del conjunto.

Esto significa que cuando existe una dependencia funcional, para poder obtener el resultado del algoritmo $f_n(f_{n-1}(x))$ es necesario esperar el resultado de $f_{n-1}(x)$, lo que implica que este tipo de algoritmos no es candidato a ser programado en paralelo.

## Ley de Amdahl

Sea $f$ la fracción de operaciones en un calculo computacional que será llevado a cabo de manera secuencial, donde $0\leq f\leq1$.

La máxima velocidad $\Psi$ alcanzada mediante programación en paralelo con una computadora con $p$ unidades de procesamiento enfocadas en el mismo calculo es:

$$\Psi\leq\frac{1}{f+(1-f)/p}$$

# Sintaxis *Numba*

Al hacer uso de *Numba* (*CUDA*), debemos tomar en cuenta que la finalidad de *Numba* es acelerar el desempeño de nuestros algoritmos.

Para llevar a cabo tal propósito, *Numba* provee de sentencias que se agregan al código de *Python* conocidos como *decoradores*. Estos decoradores son de diferentes tipos, ya que *Numba* permite acelerar el desempeño de diferentes formas.

Mediante *Numba* los cálculos de nuestros algoritmos se pueden distribuir entre el *CPU* y el *GPU*, es por eso que *Numba* nos ofrece una forma para distinguir cuáles de nuestras funciones se ejecutan en uno u otro dispositivo de cómputo.

### `nopython=True`

El decorador más sencillo de usar en *Numba* es el decorador `@jit` (*Just in time*) y permite recibir múltiples parámetros que indican la forma en la que se quiere optimizar el código (ver referencias). Si se usa este decorador sin parámetros *Numba* decide cuál es la mejor forma para optimizar el desempeño del algoritmo.

Uno de sus parámetros más elementales pero al mismo tiempo uno de los más usados, es el parametro `nopython=True`.

Al hacer uso de este decorador se le indica a *Numba* que el código que pertenece a esta definición debe ser optimizado y esto se lleva a cabo, convirtiendo el código de python en **código de máquina**, esto toma un poco más de tiempo la primera vez que se ejecuta, pero las siguientes veces tomará mucho menos tiempo que una función en *Python* nativo.

In [ ]:
import random
# biblioteca de Numba
from numba import jit

#¡¡¡descomentar para optimizar, te vas a sorprender!!!
@jit(nopython=True)
def mc_pi_aprox(n=100000000):
    dentro_circulo = 0
    for i in range(n):
      x = random.random()
      y = random.random()
      # valores dentro de la circunferencia
      if (x**2+y**2 < 1):
          dentro_circulo += 1
    return 4*dentro_circulo / n

In [ ]:
%%time
mc_pi_aprox()

CPU times: user 1.42 s, sys: 6.28 ms, total: 1.42 s
Wall time: 1.42 s


3.14175072

En el ejemplo de la celda superior en la cual se aproxima el valor de $\pi$ mediante Monte Carlo, al ejecutarlo sin el uso de *Numba* (solo *Python* nativo), podemos notar que el tiempo de ejecución se incrementa de manera proporcional a la precisión que usemos.

Por otro lado, si se usa el decorador *@jit(nopython=True)*, qué es equivalente a *@njit*, de manera explícita se le pide a *Numba*, que convierta el **código a código de máquina**. Y el tiempo de ejecución disminuye de manera notable.

### `parallel` (*OpenMP*)

Otro parámetro muy útil pero a la vez 'obscuro' es, `@njit(parrallel=True)`.

Este decorador va de la mano de la palabra reservada `prange` y en conjunto permiten ejecutar en paralelo ciclos dentro de la función definida.

Este decorador oculta mucho del proceso que se realiza al ejecutar un algoritmo en paralelo. Sin embargo ya que a esta altura del curso se conoce cual es el trasfondo (*OpenMP, MPI, CUDA*), podemos obviar el mismo.

La parlabra reservada `prange` se emplea para especificar el ciclo que se quiere realizar en paralelo y no solo eso, también realiza la operación conocida como *reduction* de alguna variable.

In [ ]:
# import de biblioteca calculos numericos
import math

# aproximacion de Pi mediante sumas de Reimann (version secuencial)
def aproxPIsec(n=10000000):
    suma=0
    # funcion a integrar
    f = lambda x: math.sqrt(1-x**2)
    delta = 1/n
    for i in range(1,n+1):
        #x = delta*(i-0.5) #punto medio
        x = delta*(i)
        suma += f(x)
    aproximacion = delta*suma*4
    return aproximacion

In [ ]:
%%time
aproxPIsec()

CPU times: user 3.61 s, sys: 10.9 ms, total: 3.62 s
Wall time: 3.63 s


3.1415924535527124

Usamos la 'función mágica' `%%time` para mostrar el tiempo y el resultado de la ejecución de la celda en la que se hace uso de la versión secuencial de este algoritmo.

In [ ]:
# import de biblioteca calculos numericos y paralelizacion
from numba import njit, prange, set_num_threads
import numpy as np
import math
# fijamos desde código el numero de hilos
set_num_threads(2)

# aproximacion de Pi mediante sumas de Reimann (version paralela)
@njit(parallel=True)
def aproxPIpar(n=10000000):
    suma=0
    # funcion a integrar
    f = lambda x: math.sqrt(1-x**2)
    delta = 1/n
    # prange indica que este for debe ser ejecutado en paralelo
    for i in prange(1,n+1):
        #x = delta*(i-0.5) #punto medio
        x = delta*(i)
        suma += f(x)
    aproximacion = delta*suma*4
    return aproximacion

In [ ]:
%%time
aproxPIpar()

CPU times: user 446 ms, sys: 2.74 ms, total: 449 ms
Wall time: 426 ms


3.1415924535525024

#### Versión en paralelo

Hay varios puntos importantes a notar en la versión paralelo del algoritmo de la aproximación del valor de $\pi$ mediante sumas de Raimann:

1.   En la sección de imports, `njit`, `prange`, `set_num_threads`, estos son los elementos de *Numba* que nos permiten la ejecución de este algoritmo en paralelo.
2.   El decorador `@njit(parallel=True)` en conjunto con `prange`, nos indican que secciones del código se ejecutará en paralelo.
3.   *Numba* oculta la operación de *redcution sum*, de la variable `suma`.
4.   El tiempo de ejecución disminuye bastante en la versión en paralelo con respecto a la versión secuencial y no así la precisión, de hecho podemos descomentar la linea inmediata debajo del ciclo `for` para mejorar la precisión del algoritmo al usar el punto medio.



# Referencias

1.   https://numba.pydata.org/numba-doc/latest/user/5minguide.html
2.   http://numba.pydata.org/numba-doc/latest/user/threading-layer.html
3.   https://numba.pydata.org/numba-doc/dev/user/jit.html
4.   https://thedatafrog.com/en/articles/make-python-fast-numba/
5.   https://people.duke.edu/~ccc14/sta-663/CUDAPython.html





